# Fiddling around with making Bungie API calls to get json data 

## [Error Codes](https://bungie-net.github.io/multi/schema_Exceptions-PlatformErrorCodes.html#schema_Exceptions-PlatformErrorCodes)
## [Manifest](https://bungie-net.github.io/multi/operation_get_Destiny2-GetDestinyManifest.html#operation_get_Destiny2-GetDestinyManifest)


In [2]:
import pandas as pd
import requests
import os

In [18]:
class BungieData(object):
    '''
    For the API calls below, no authentication is needed. You'll just need to have your Bungie API key exported in your bash profile
    and named as BUNGIE_API_KEY to run the script as-is.
    '''

    def __init__(self, api_key):
        '''
        api_key (str): The api key given to you by Bungie when you registered your app with them
        '''
        self.api_key = api_key

    def get_playerByTagName(self, gamertag):
        '''gamertag (str): The PSN gamertag a player uses on Destiny 2'''
        site_call = "https://bungie.net/Platform/Destiny2/SearchDestinyPlayer/2/" + gamertag
        request = requests.get(site_call,
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']

    def get_DestinyUserId(self, gamertag):
        '''gamertag (str): The PSN gamertag a player uses on Destiny 2'''
        info = self.get_playerByTagName(gamertag)
        return int(info[0]['membershipId'])

    def get_BungieUserId(self, membership_id):
        '''
        membership_id (int): the Destiny membership_id of a player (the id returned by get_DestinyUserId)
        Uses old Destiny endpoint for a PSN user to get the BUNGIE membershipId
        '''
        site_call = "https://bungie.net/Platform/User/GetMembershipsById/" + str(membership_id) + "/2/"
        request = requests.get(site_call,
                                headers={"X-API-Key":self.api_key})
        return int(request.json()['Response']['bungieNetUser']['membershipId'])

    def get_DestinyUserProfile(self, membership_id, components=[100]):
        '''
        membership_id (int): the Destiny membership_id of a player (returned by get_DestinyUserId)
        components (list of ints): the type of info you want returned according the Bungie API docs.
          Defaults to 100: basic profile info ([100, 200] would also return more detailed info by Destiny character
        Uses new Destiny 2 endpoint for PSN player using the Destiny membershipId
        '''
        components = "?components=" + ','.join([str(c) for c in components])
        site_call = "https://bungie.net/Platform/Destiny2/2/Profile/" + str(membership_id) + "/" + components
        request = requests.get(site_call,
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']

    def get_postGameStats(self, game_id):
        '''game_id (int): Need to look further into this, but game_ids can be found'''
        site_call = "https://bungie.net/Platform/Destiny2/Stats/PostGameCarnageReport/" + str(game_id)
        request = requests.get(site_call,
                                headers={"X-API-Key":self.api_key})
        return request.json()

    def get_Manifest(self):
        site_call = "https://bungie.net/Platform/Destiny2/Manifest/"
        request = requests.get(site_call,
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']

    def get_PlayerStats(self, membership_id):
        site_call = "https://bungie.net/Platform/Destiny2/2/Account/" + str(membership_id) + "/Stats/"
        request = requests.get(site_call,
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']

    def get_StatDefinitions(self):
        site_call = "https://bungie.net/Platform/Destiny2/Stats/Definition/"
        request = requests.get(site_call,
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_GameIdsByCharacter(self, destiny_id, character_id):
        site_call = "https://bungie.net/Platform/Destiny2/2/Profile/" + str(destiny_id) + "/Character/" + str(character_id) + "/?components=CharacterActivities"
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    

In [19]:
bungie = BungieData(api_key=os.environ["BUNGIE_API_KEY"]) # Never put your keys in code... export 'em!

In [5]:
# Get my own Bungie MembershipID
Me = bungie.get_playerByTagName("BalancedSeeker6")
Me

[{u'displayName': u'BalancedSeeker6',
  u'iconPath': u'/img/theme/destiny/icons/icon_psn.png',
  u'membershipId': u'4611686018453385415',
  u'membershipType': 2}]

In [6]:
bungie = BungieData(api_key=os.environ["BUNGIE_API_KEY"])

4611686018453385415

In [7]:
# Get my BUNGIE membershipID from using my DESTINY membershipID found in the above call
# This is the number found on the profile page of a user on Bungie.net
my_bungie_id = bungie.get_BungieUserId(4611686018453385415)
my_bungie_id

11927477

### To use GetProfile, you'll need to lookup the user's Destiny membershipID by searching by gamertag. Found that in the get_DestinyUserId call : 4611686018453385415

In [8]:
My_Profile = bungie.get_DestinyUserProfile(4611686018453385415, components=[100])
My_Profile 

{u'itemComponents': {},
 u'profile': {u'data': {u'characterIds': [u'2305843009260518141',
    u'2305843009289675361'],
   u'dateLastPlayed': u'2017-10-03T18:18:26Z',
   u'userInfo': {u'displayName': u'BalancedSeeker6',
    u'membershipId': u'4611686018453385415',
    u'membershipType': 2},
   u'versionsOwned': 1},
  u'privacy': 1}}

In [9]:
# Get a random single game's stats
game_stats = bungie.get_postGameStats(100)
game_stats

{u'activityDetails': {u'directorActivityHash': 1658347443,
  u'instanceId': u'100',
  u'isPrivate': False,
  u'mode': 2,
  u'modes': [7, 2],
  u'referenceId': 1658347443},
 u'entries': [{u'characterId': u'2305843009260533952',
   u'extended': {u'values': {u'precisionKills': {u'basic': {u'displayValue': u'55',
       u'value': 55.0}}},
    u'weapons': [{u'referenceId': 53159280,
      u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'4',
         u'value': 4.0}},
       u'uniqueWeaponKillsPrecisionKills': {u'basic': {u'displayValue': u'75%',
         u'value': 0.75}},
       u'uniqueWeaponPrecisionKills': {u'basic': {u'displayValue': u'3',
         u'value': 3.0}}}},
     {u'referenceId': 4024037919,
      u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'75',
         u'value': 75.0}},
       u'uniqueWeaponKillsPrecisionKills': {u'basic': {u'displayValue': u'65%',
         u'value': 0.6533333333333333}},
       u'uniqueWeaponPrecisionKills': {u'basic': {u'

In [10]:
manifest = bungie.get_Manifest()
manifest

{u'mobileAssetContentPath': u'/common/destiny2_content/sqlite/asset/asset_sql_content_30228cabcdcf681095c36fab1da69e32.content',
 u'mobileClanBannerDatabasePath': u'/common/destiny2_content/clanbanner/clanbanner_sql_content_12a291b583d3f5ce5c30a648fa58b482.content',
 u'mobileGearAssetDataBases': [{u'path': u'/common/destiny2_content/sqlite/asset/asset_sql_content_30228cabcdcf681095c36fab1da69e32.content',
   u'version': 0},
  {u'path': u'/common/destiny2_content/sqlite/asset/asset_sql_content_669cef97d6bb31f46d5703bcb9acb7bd.content',
   u'version': 1},
  {u'path': u'/common/destiny2_content/sqlite/asset/asset_sql_content_669cef97d6bb31f46d5703bcb9acb7bd.content',
   u'version': 2}],
 u'mobileGearCDN': {u'Gear': u'/common/destiny2_content/geometry/gear',
  u'Geometry': u'/common/destiny2_content/geometry/platform/mobile/geometry',
  u'PlateRegion': u'/common/destiny2_content/geometry/platform/mobile/plated_textures',
  u'Shader': u'/common/destiny2_content/geometry/platform/mobile/shad

In [11]:
My_Stats = bungie.get_PlayerStats(my_destiny_id)
My_Stats

{u'characters': [{u'characterId': u'2305843009289675361',
   u'deleted': False,
   u'merged': {u'allTime': {u'activitiesCleared': {u'basic': {u'displayValue': u'17',
       u'value': 17.0},
      u'statId': u'activitiesCleared'},
     u'activitiesEntered': {u'basic': {u'displayValue': u'47', u'value': 47.0},
      u'statId': u'activitiesEntered'},
     u'activitiesWon': {u'basic': {u'displayValue': u'1', u'value': 1.0},
      u'statId': u'activitiesWon'},
     u'adventuresCompleted': {u'basic': {u'displayValue': u'3', u'value': 3.0},
      u'pga': {u'displayValue': u'0.1', u'value': 0.06382978723404255},
      u'statId': u'adventuresCompleted'},
     u'allParticipantsCount': {u'basic': {u'displayValue': u'230',
       u'value': 230.0},
      u'statId': u'allParticipantsCount'},
     u'allParticipantsScore': {u'basic': {u'displayValue': u'335',
       u'value': 335.0},
      u'statId': u'allParticipantsScore'},
     u'allParticipantsTimePlayed': {u'basic': {u'displayValue': u'3d 7h',
  

In [12]:
def char_stats_json2df(json):
    for char in My_Stats['characters']:
        if not char['deleted']:
            char_id = int(char['characterId'])

statsdf = pd.DataFrame(My_Stats)
statsdf

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
stat_definitions = bungie.get_StatDefinitions()
stat_definitions

In [16]:
activities = bungie.get_GameIdsByCharacter(4611686018453385415, 2305843009260518141)
activities

{u'activities': {u'data': {u'availableActivities': [{u'activityHash': 1290744998,
     u'canJoin': True,
     u'canLead': True,
     u'difficultyTier': 2,
     u'isCompleted': False,
     u'isNew': False,
     u'isVisible': True,
     u'recommendedLight': 10},
    {u'activityHash': 3243161126,
     u'canJoin': True,
     u'canLead': True,
     u'difficultyTier': 2,
     u'isCompleted': True,
     u'isNew': False,
     u'isVisible': True,
     u'recommendedLight': 10},
    {u'activityHash': 3062197616,
     u'canJoin': True,
     u'canLead': True,
     u'difficultyTier': 2,
     u'isCompleted': True,
     u'isNew': False,
     u'isVisible': True,
     u'recommendedLight': 10},
    {u'activityHash': 1975064760,
     u'canJoin': True,
     u'canLead': True,
     u'difficultyTier': 2,
     u'displayLevel': 24,
     u'isCompleted': False,
     u'isNew': False,
     u'isVisible': True,
     u'recommendedLight': 240},
    {u'activityHash': 1863334927,
     u'canJoin': True,
     u'canLead': T

In [20]:
game_stats = bungie.get_postGameStats(1984315274)
game_stats

{u'ErrorCode': 1653,
 u'ErrorStatus': u'DestinyPGCRNotFound',
 u'Message': u'The activity you were looking for was not found.',
 u'MessageData': {},
 u'ThrottleSeconds': 0}

# Using get_data.py as a module


In [29]:
import os
import sys

# Solves the relative, adjacent directory import problem
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.get_data import BungieData

In [30]:
bungie = BungieData(api_key=os.environ["BUNGIE_API_KEY"], membershipType=2)

In [31]:
# Get my own Bungie MembershipID
Me = bungie.get_playerByTagName("BalancedSeeker6")
Me

[{u'displayName': u'BalancedSeeker6',
  u'iconPath': u'/img/theme/destiny/icons/icon_psn.png',
  u'membershipId': u'4611686018453385415',
  u'membershipType': 2}]

In [34]:
bungie.get_DestinyUserId("BalancedSeeker6")

4611686018453385415

In [28]:
characters = bungie.get_DestinyUserProfile(4611686018453385415)
characters['profile']['data']['characterIds']

[u'2305843009260518141', u'2305843009289675361', u'2305843009303434294']

In [33]:
character_list = bungie.get_CharacterIdsByPlayer(4611686018453385415)
character_list

[u'2305843009260518141', u'2305843009289675361', u'2305843009303434294']

In [ ]:
for char in character_list:
    pass